# Магические методы классов

## Строковое представление объектов

- `__repr__` - строковое представление объектов
- `__str__` - метод, который вызывается функциями `str`, `format`, `print`
- `__format__` - метод, который вызывается при форматировании строки (`.format(...)`, f-strings)

In [2]:
class A:
    def __init__(self, value):
        self.value = value
    
    def __str__(self):
        return f'A({self.value}) object str method'
    
    def __repr__(self):
        return f'A({self.value}) object repr method'
    
    def __format__(self, format_spec):
        return f'A({self.value}) object format method'
    

a = A(365)
print(a)
print("some text with %s" % a)
print(f"some text with {a}")
print("some text with {a}".format(a=a))
a

A(365) object str method
some text with A(365) object str method
some text with A(365) object format method
some text with A(365) object format method


A(365) object repr method

## Rich Comparison

Магические методы для использования объекта класса с операторами сравнения. Нужно либо указать все, либо использовать декоратор total_ordering, который дополнит недостающие методы "антонимами":

In [3]:
from functools import total_ordering

@total_ordering
class Person(object):

    def __init__(self, firstname, lastname):
        self.first = firstname
        self.last = lastname

    def __eq__(self, other):
        print(f"{self.first} {self.last} eq:", end='\t')
        return ((self.last, self.first) == (other.last, other.first))

#     def __ne__(self, other):
#         print(f"{self.first} {self.last} ne:", end='\t')
#         return ((self.last, self.first) != (other.last, other.first))

#     def __lt__(self, other):
#         print(f"{self.first} {self.last} lt:", end='\t')
#         return ((self.last, self.first) < (other.last, other.first))

    def __le__(self, other):
        print(f"{self.first} {self.last} le:", end='\t')
        return ((self.last, self.first) <= (other.last, other.first))

#     def __gt__(self, other):
#         print(f"{self.first} {self.last} gt:", end='\t')
#         return ((self.last, self.first) > (other.last, other.first))

    def __ge__(self, other):
        print(f"{self.first} {self.last} ge:", end='\t')
        return ((self.last, self.first) >= (other.last, other.first))

    def __repr__(self):
        return "%s %s" % (self.first, self.last)
    
    
a = Person("Евпатий", "Коловрат")
b = Person("Илья", "Муромец")

print(a == b)
print(a != b)
print(a >= b)
print(a > b)
print(a <= b)
print(a < b)

Евпатий Коловрат eq:	False
Евпатий Коловрат eq:	True
Евпатий Коловрат ge:	False
Евпатий Коловрат le:	False
Евпатий Коловрат le:	True
Евпатий Коловрат le:	Евпатий Коловрат eq:	True


### Метод `__hash__`

Этот метод отвечает за уникальность объекта и возможность его использования как ключ в hashable-коллекциях. Создадим пустой класс и посмотрим, как в нем работают оператор `is`, использование его как ключа в hashable-коллекции и сравнение.

In [4]:
class X:
    pass

a = X()
b = X()

print(a is b)
print({a: 1, b: 2})
print(a == b)
print(hash(a), hash(b))

False
{<__main__.X object at 0x000002B6B7EF9010>: 1, <__main__.X object at 0x000002B6B7CFE990>: 2}
False
186487077121 186486947481


In [5]:
c = a
d = {a: 1, b: 2}
d[c]

1

In [6]:
c in [a, b]

True

Добавим метод, который делает наш объект равным чему угодно.

In [7]:
class X:
    def __eq__(self, other):
        return True


a = X()
b = X()

print(a == b)
print(a is b)
# print({a: 1, b: 2})
print(hash(a), hash(b))

True
False


TypeError: unhashable type: 'X'

Взять хеш от объектов не получилось, поскольку при определенном методе `__eq__` сравнение объектов по уникальности неочевидно для интерпретатора. Необходимо реализовать собственный метод `__hash__` в классе. Основное свойство хеша состоит в том, что он должен быть разным для разных объектов. Но что будет, если не придерживаться этого свойства? 

In [8]:
class X:
    def __eq__(self, other):
        return True
    
    def __hash__(self):
        return 25
    
a = X()
b = X()

print(a == b)
print(a is b)
print({a: 1, b: 2})
print(hash(a), hash(b))

True
False
{<__main__.X object at 0x000002B6B7EFA3C0>: 2}
25 25


Значение в словаре перезаписалось, поскольку хеш от обоих объектов одинаковый. Оператор `is` тем не менее выдает False, поскольку id объектов разные.

## Работа с числовыми операторами

https://habr.com/ru/post/186608/

### Преобразование типов
- `__int__(self)` - Преобразование типа в int.
- `__long__(self)` - Преобразование типа в long.
- `__float__(self)` - Преобразование типа в float.
- `__complex__(self)` - Преобразование типа в комплексное число.
- `__oct__(self)` - Преобразование типа в восьмеричное число.
- `__hex__(self)` - Преобразование типа в шестнадцатиричное число.
- `__index__(self)` - Преобразование типа к int, когда объект используется в срезах (выражения вида `[start:stop:step]`). Если вы определяете свой числовый тип, который может использоваться как индекс списка, вы должны определить `__index__`.
- `__trunc__(self)` - Вызывается при math.trunc(self). Должен вернуть своё значение, обрезанное до целочисленного типа (обычно long).


### Унарные операции

- `__pos__(self)` - Определяет поведение для унарного плюса (`+some_object`)
- `__neg__(self)` - Определяет поведение для отрицания (`-some_object`)
- `__abs__(self)` - Определяет поведение для встроенной функции abs().
- `__invert__(self)` - Определяет поведение для бинарного инвертирования оператором ~.
- `__round__(self, n)` - Определяет поведение для встроенной функции round(). n - это число знаков после запятой, до которого округлить.
- `__floor__(self)` - Определяет поведение для math.floor(), то есть, округления до ближайшего меньшего целого.
- `__ceil__(self)` - Определяет поведение для math.ceil(), то есть, округления до ближайшего большего целого.
- `__trunc__(self)` - Определяет поведение для math.trunc(), то есть, обрезания до целого.


### Бинарные операции


Следующие операции вызываются над левым операндом в операторах. Т.е., например, `x + y` - это `x.__add__(y)`.

- `__add__(self, other)` - Сложение.
- `__sub__(self, other)` - Вычитание.
- `__mul__(self, other)` - Умножение.
- `__matmul__(self, other)` - Оператор перемножения матриц `@`
- `__floordiv__(self, other)` - Целочисленное деление, оператор `//`.
- `__truediv__(self, other)` - Деление, оператор `/`.
- `__mod__(self, other)` - Остаток от деления, оператор %.
- `__divmod__(self, other)` - Определяет поведение для встроенной функции divmod().
- `__pow__(self, other)` - Возведение в степень, оператор `**`.
- `__lshift__(self, other)` - Двоичный сдвиг влево, оператор `<<`.
- `__rshift__(self, other)` - Двоичный сдвиг вправо, оператор `>>`.
- `__and__(self, other)` - Двоичное И, оператор `&`.
- `__or__(self, other)` - Двоичное ИЛИ, оператор `|`.
- `__xor__(self, other)` - Двоичный xor, оператор `^`.

Кроме того, есть функции, с префиксом `r`, например, `__radd__`. Это аналоги тех же функций, но только для тех случаев, когда операнд стоит справа. Т.е. `x + y` - это `y.__radd__(x)`.

И также есть префикс `i`, например, `__iadd__`, для случаев операций с присвоением (`x += y`).

In [9]:
class Meal:
    def __init__(self, title, price):
        self.title = title
        self.price = price
        
    def __str__(self):
        return ': '.join([self.title, str(self.price)])
    
    def __repr__(self):
        """Функция, которая используется для текстового представления объекта в случаях, когда это происходит не
        через функцию str(obj)"""
        return str(self)
    
    def __add__(self, other):
        """Функция, которая описывает прибавление к нашему объекту объекта other"""
        # если у нас оба объекта данного класса, сложим их атрибуты
        if isinstance(other, Meal):
            new_title = ', '.join([self.title, other.title])
            new_price = self.price + other.price
        else:
            # а если второй объект не этого класса, то попробуем его привести к типу float
            new_title = self.title + " и что-то еще"
            new_price = self.price + float(other)
        return Meal(new_title, new_price)
    
    def __iadd__(self, other):
        print(f"Нельзя впихнуть в {self.title} другую еду")
        return self
    
    def __neg__(self):
        return Meal(self.title, -self.price)

In [10]:
Meal("БигМак", 200)

БигМак: 200

In [11]:
Meal("БигМак", 200) + Meal("Картошка", 50)

БигМак, Картошка: 250

In [12]:
Meal("БигМак", 200) + 25

БигМак и что-то еще: 225.0

In [13]:
25 + Meal("БигМак", 200)

TypeError: unsupported operand type(s) for +: 'int' and 'Meal'

Произошла ошибка, поскольку у нас определено сложение только <code>Meal + что-то</code>, но не <code>что-то + Meal</code>. В случаях, когда складываемые объекты разных типов, операция сложения в питоне некоммутативна. Чтобы определить обратное сложение, добавим метод <code>\__radd__</code>

In [14]:
Meal.__radd__ = Meal.__add__

25 + Meal("БигМак", 200)

БигМак и что-то еще: 225.0

In [15]:
- Meal("Еда", 2000)

Еда: -2000

In [16]:
a = Meal('доширак', 40)
a += 1000
a

Нельзя впихнуть в доширак другую еду


доширак: 40

## Эмуляция контейнеров

- `object.__len__(self)` - длина контейнера
- `object.__getitem__(self, key)` - получение элемента по индексу/ключу
- `object.__setitem__(self, key, value)` - назначение элемента
- `object.__delitem__(self, key)` - удаление элемента
- `object.__missing__(self, key)` - этот метод определен только для классов-наследников словаря и вызывается методом `__getattr__`, когда в словаре нет требуемого элемента
- `object.__iter__(self)` - получение итератора
- `object.__reversed__(self)` - обратный порядок элементов
- `object.__contains__(self, item)` - оператор in

In [17]:
class NumberDict(dict):
    def __getitem__(self, key):
        return super(NumberDict, self).__getitem__(float(key))
    
    def __setitem__(self, key, value):
        super(NumberDict, self).__setitem__(float(key), value)
        
    def __delitem__(self, key):
        super(NumberDict, self).__delitem__(float(key))
         
    def __missing__(self, key):
        return 'no-no-no'
    
    def __contains__(self, item):
        return self[item] != self.__missing__(1)


my_dict = NumberDict()  
print(my_dict["0"])

my_dict["0"] = 1
print(my_dict[0])

print(1 in my_dict)
print("0.000" in my_dict)

no-no-no
1
False
True


## Контроль объектов

- `__dir__(self)` - Определяет поведение функции `dir()`, вызванной на экземпляре вашего класса. Этот метод должен возвращать пользователю список атрибутов. Обычно, определение `__dir__` не требуется, но может быть жизненно важно для интерактивного использования вашего класса, если вы переопределили `__getattr__` или `__getattribute__` (с которыми вы встретитесь в следующей части), или каким-либо другим образом динамически создаёте атрибуты.
- `__sizeof__(self)` - Определяет поведение функции `sys.getsizeof()`, вызыванной на экземпляре вашего класса. Метод должен вернуть размер вашего объекта в байтах. Он главным образом полезен для классов, определённых в расширениях на C, но всё-равно полезно о нём знать.

Дурацкий пример `__dir__`:

In [18]:
class SlotsClass:
    __slots__ = ('foo', 'bar')
    
    def __dir__(self):
        return list(self.__slots__)
    
dir(SlotsClass())

['bar', 'foo']

## Задание

Напишем свой аналог листа таблицы Excel. Нужно написать структуру данных `Field`, в которой доступ к значениям будет осуществляться по ключам. Ключом будет пара "буква" - "число", по аналогии с адресом ячейки в Excel. Возможные форматы обращения к одной и той же "ячейке" данных:

`field = Field()`
- `field[1, 'a'] = 25`
- `field['a', 1] = 25`
- `field['a', '1'] = 25`
- `field['1', 'a'] = 25`
- `field['1a'] = 25`
- `field['a1'] = 25`
- `field[1, 'A'] = 25`
- `field['A', 1] = 25`
- `field['A', '1'] = 25`
- `field['1', 'A'] = 25`
- `field['1A'] = 25`
- `field['A1'] = 25`

В этом списке каждая из этих строк записывает число `25` в ячейку с одним и тем же ключом. Соответственно, по любому из перечисленных ключей должно быть можно получить это число из объекта `field`. Также должны быть реализованы удаление элемента из структуры и возможность использования оператора `in`, например:

- `(1, 'a') in field`: True
- `"A1" in field`: True
- `('D', '4') in field`: False

Таким образом, выходит, что ключом структуры может быть либо кортеж, либо строка. При попытке получить или записать значение по ключу другого типа должно быть вызвано исключение `TypeError`. При некорректном значении строки или элементов кортежа нужно вызывать исключение `ValueError`. Корректными значениями будет считать одиночные буквы и неотрицательное целое число любой длины, т.е. правильные варианты ключей:

- А1
- А222543
- Z89

Неправильные варианты ключей:

- AA5
- Q2.5
- -6F
- A
- 27
- GG

Кроме вышеперчисленного, по объекту должно быть возможно итерироваться. При проходе циклом по объекту должны возвращаться _значения_, хранящиеся в нём. Порядок возврата значений не важен.

### Подсказка

В своем решении этого задания я использовал в качестве ключей хранимого словаря frozenset, а проверку на ValueError реализовал через регулярку. Также рекомендую проверку типов и преобразование поступившего ключа в тот вид, в котором он хранится "под капотом", вынести в отдельный метод и вызывать его из всех описываемых магических методов.

In [38]:
import re

class Field:
    def __init__(self):
        self.data = {}

    def _normalize_key(self, key):
        if isinstance(key, tuple) and len(key) == 2:
            row, col = key

            # Handle cases like ('a', '1')
            if isinstance(row, str) and isinstance(col, str):
                if row.isalpha() and col.isdigit():
                    return (int(col), row.lower())
                elif row.isdigit() and col.isalpha():
                    return (int(row), col.lower())
                else:
                    raise ValueError("Invalid tuple key format")

            # Handle cases like (1, 'a') or ('A', 1)
            if isinstance(row, int) and isinstance(col, str) and len(col) == 1 and col.isalpha():
                return (row, col.lower())
            elif isinstance(row, str) and row.isdigit() and isinstance(col, int):
                return (int(row), chr(col + ord('a')))
            elif isinstance(row, str) and len(row) == 1 and row.isalpha() and isinstance(col, int):
                return (col, row.lower())
            else:
                raise ValueError("Invalid tuple key format")
        
        elif isinstance(key, str):
            # Handle formats like 'A1' or '1A' НЕ РАБОТАЕТ
            match = re.match(r'^([A-Za-z]+)(\d+)$', key)  # Letters followed by digits
            if match:
                col, row = match.groups()
                if len(col) != 1:
                    raise ValueError("Column must be a single letter")
                return (int(row), col.lower())
            
            match = re.match(r'^(\d+)([A-Za-z]+)$', key)  # Digits followed by letters
            if match:
                row, col = match.groups()
                if len(col) != 1:
                    raise ValueError("Column must be a single letter")
                return (int(row), col.lower())
            
            raise ValueError("Invalid string key format")
        
        raise TypeError("Key must be either a tuple or a string")

    def __setitem__(self, key, value):
        try:
            normalized_key = self._normalize_key(key)
            self.data[normalized_key] = value
        except (ValueError, TypeError) as e:
            raise KeyError(f"Invalid key: {key}. Error: {e}")

    def __getitem__(self, key):
        try:
            normalized_key = self._normalize_key(key)
            return self.data[normalized_key]
        except (KeyError, ValueError, TypeError) as e:
            raise KeyError(f"Invalid key: {key}. Error: {e}")

    def __delitem__(self, key):
        try:
            normalized_key = self._normalize_key(key)
            del self.data[normalized_key]
        except (KeyError, ValueError, TypeError) as e:
            raise KeyError(f"Invalid key: {key}. Error: {e}")

    def __contains__(self, key):
        try:
            normalized_key = self._normalize_key(key)
            return normalized_key in self.data
        except (ValueError, TypeError) as e:
            return False

    def __iter__(self):
        return iter(self.data.items())

# Пример использования
def test_field():
    field = Field()

    # Проверка установки значений
    field[1, 'a'] = 25
    field['A', 1] = 25
    field['a', '1'] = 25
    field['1', 'a'] = 25
    field['1a'] = 25
    field['a1'] = 25
    field[1, 'A'] = 25
    field['A', 1] = 25

    # Проверка получения значений
    print(field[1, 'a'])  # Output: 25
    print(field['A', 1])  # Output: 25
    print(field['a', '1'])  # Output: 25
    print(field['1', 'a'])  # Output: 25
    print(field['1a'])  # Output: 25
    print(field['a1'])  # Output: 25
    print(field[1, 'A'])  # Output: 25
    print(field['A', 1])  # Output: 25

    # Проверка удаления значений
    del field[1, 'a']
    print((1, 'a') in field)  # Output: False

    # Проверка оператора in
    print((1, 'a') in field)  # Output: False
    print('A1' in field)  # Output: True
    print(('D', '4') in field)  # Output: False

    # Проверка итерации
    for key, value in field:
        print(f"{key}: {value}")

test_field()

25
25
25
25
25
25
25
25
False
False
False
False
